In [1]:
from pymongo import MongoClient
import yfinance as yf

In [2]:
client = MongoClient("mongodb://localhost:27017")
db = client['new_db']
presentation_collection = db['presentation_news']
stock_ticker_presentation_collection = db['stock_ticker_presentation']

In [3]:
presentation_data_list = list(presentation_collection.find())
stock_ticker_data_list = list(stock_ticker_presentation_collection.find())

In [4]:
len(stock_ticker_data_list)

182

In [5]:
ticker_list  = []
for stock in stock_ticker_data_list:
    # stock_data = stock[1]
    for key, value in stock.items():
        ticker_list.append(value)

In [6]:
from bson import ObjectId
final_ticker_list = [ticker_name for ticker_name in ticker_list if not isinstance(ticker_name, ObjectId)]

In [7]:
final_ticker_list[:-1]

['WCIL.NS',
 'CELLO.NS',
 'COALINDIA.NS',
 'PRICOLLTD.NS',
 'SHIVATEX.NS',
 'EICHERMOT.NS',
 'RITESHIN.BO',
 'GEOJITFSL.NS',
 'RAILTEL.NS',
 'SIGNATURE.NS',
 'SKYGOLD.NS',
 'GOACARBON.BO',
 'LT.NS',
 'SUNPHARMA.NS',
 'PGIL.NS',
 'APLAPOLLO.NS',
 'KPIL.NS',
 'POLYCAB.NS',
 'METROBRAND.BO',
 'LUPIN.NS',
 'SHRIRAMPPS.NS',
 'JUBLFOOD.NS',
 'ITC.NS',
 'WAAREEENER.BO',
 'ICICIPRULI.BO',
 'SUZLON.NS',
 'IPCALAB.NS',
 'KEI.NS',
 'WELSPUNLIV.NS',
 'JSWENERGY.NS',
 'MAZDOCK.BO',
 'HOMEFIRST.NS',
 'JSWINFRA.BO',
 'PETRONET.NS',
 'INDUSINDBK.NS',
 'EPACK.NS',
 'RAYMONDLSL.NS',
 'HDFCBANK.NS',
 'RAYMONDLSL.NS',
 'PANACEABIO.NS',
 'BPCL.NS',
 'RAMKY.BO',
 'HGINFRA.NS',
 'VEDL.NS',
 'LT.NS',
 'HGINFRA.NS',
 'HAL.NS',
 'PNBHOUSING.BO',
 'HDFCBANK.NS',
 'NTPCGREEN.BO',
 'AUROPHARMA.NS',
 'IBN',
 'AVANTIFEED.NS',
 'ANANTRAJ.BO',
 'DEVYANI.BO',
 'JSWINFRA.BO',
 'TATAMOTORS.NS',
 'SPICEJET.BO',
 'MTARTECH.NS',
 'ZODIAC.NS',
 'LUMAXTECH.BO',
 'SPICEJET.BO',
 'MTARTECH.NS',
 'ZODIAC.NS',
 'LUMAXTECH.BO',
 '

In [152]:
# ticker_list

In [153]:
len(presentation_data_list)

182

<h3>Taking historical data</h3>

In [154]:
import numpy as np

stock_tickers = final_ticker_list.copy()

embedding_dim = 16  

stock_embeddings = {}

for ticker in stock_tickers:
    stock_embeddings[ticker] = np.random.rand(embedding_dim)


print(f"Embedding for 'WCIL.NS': {stock_embeddings['WCIL.NS']}")


Embedding for 'WCIL.NS': [0.24520364 0.39925635 0.13053077 0.61538465 0.0697574  0.86128406
 0.92685906 0.23682762 0.95212294 0.28251007 0.95847077 0.93948136
 0.51406995 0.66161203 0.95780376 0.45381559]


In [155]:
print(f" {stock_embeddings['KPIL.NS']}")

 [0.31149451 0.96898494 0.979234   0.13864564 0.17541962 0.51071561
 0.81375833 0.57924885 0.68805311 0.61018044 0.73203585 0.18679368
 0.49499438 0.52120132 0.24953592 0.60597807]


In [156]:
# stock_embeddings

historical_data

In [157]:
def fetch_stock_data(tickers, start_date, end_date):
    stock_data = {}
    for ticker in tickers:
        # print(f"Fetching data for {ticker}...")
        data = yf.download(ticker, start=start_date, end=end_date)
        stock_data[ticker] = data
    return stock_data

# Example: Fetch data for the last 10 years
start_date = "2014-01-01"
end_date = "2025-01-01"
data = fetch_stock_data(stock_tickers, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [158]:
from sklearn.preprocessing import MinMaxScaler


def preprocess_stock_data(stock_data, sequence_length=30):
    scaler = MinMaxScaler()
    stock_data_scaled = scaler.fit_transform(stock_data[['Open', 'High', 'Low', 'Close', 'Volume']])
    
    X, y = [], []
    for i in range(sequence_length, len(stock_data_scaled)):
        X.append(stock_data_scaled[i-sequence_length:i])
        y.append(stock_data_scaled[i, 3])  
    return np.array(X), np.array(y)



sequence_length = 30
X_all, y_all, stock_ids = [], [], []
for i, ticker in enumerate(stock_tickers):
    if ticker in data:
        stock_data = data[ticker].dropna()  
        X, y = preprocess_stock_data(stock_data, sequence_length)
        
        if X.shape[0] > 0 and X.ndim == 3 and y.ndim == 1:
            X_all.append(X)
            y_all.append(y)
            stock_ids.extend([i] * len(X))


X_all = np.vstack(X_all)
y_all = np.hstack(y_all)
stock_ids = np.array(stock_ids)


In [159]:

embedding_dim = 16
stock_embeddings = np.random.rand(len(stock_tickers), embedding_dim)

stock_embedding_data = stock_embeddings[stock_ids]


In [160]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input

# Model Inputs
historical_input = Input(shape=(sequence_length, 5), name="historical_data")  # 5 features (OHLCV)
embedding_input = Input(shape=(embedding_dim,), name="stock_embedding")

# LSTM for Historical Data
x = layers.LSTM(64, return_sequences=True)(historical_input)
x = layers.LSTM(32)(x)

# Combine LSTM output with Stock Embedding
x = layers.concatenate([x, embedding_input])
x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)

# Output Layer (Regression for 'Close' price)
output = layers.Dense(1, activation="linear", name="output")(x)

# Model
model = models.Model(inputs=[historical_input, embedding_input], outputs=output)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ historical_data     │ (None, 30, 5)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 30, 64)    │     17,920 │ historical_data[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 32)        │     12,416 │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stock_embedding     │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 48)        │          0 │ lstm_3[0][0],     │
│ (Concatenate)       │                   │            │ stock_embedding[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      3,136 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      2,080 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         33 │ dense_3[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 35,585 (139.00 KB)

 Trainable params: 35,585 (139.00 KB)

 Non-trainable params: 0 (0.00 B)

In [161]:

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test, embeddings_train, embeddings_test = train_test_split(
    X_all, y_all, stock_embedding_data, test_size=0.2, random_state=42
)


model.fit(
    [X_train, embeddings_train],
    y_train,
    validation_data=([X_test, embeddings_test], y_test),
    epochs=10,
    batch_size=64
)


Epoch 1/10
4347/4347 ━━━━━━━━━━━━━━━━━━━━ 198s 43ms/step - loss: 0.0020 - mae: 0.0225 - val_loss: 4.0510e-04 - val_mae: 0.0148
Epoch 2/10
4347/4347 ━━━━━━━━━━━━━━━━━━━━ 162s 37ms/step - loss: 2.2117e-04 - mae: 0.0097 - val_loss: 1.6662e-04 - val_mae: 0.0078
Epoch 3/10
4347/4347 ━━━━━━━━━━━━━━━━━━━━ 234s 54ms/step - loss: 1.9946e-04 - mae: 0.0089 - val_loss: 1.6789e-04 - val_mae: 0.0080
Epoch 4/10
4347/4347 ━━━━━━━━━━━━━━━━━━━━ 241s 55ms/step - loss: 1.8531e-04 - mae: 0.0085 - val_loss: 1.5531e-04 - val_mae: 0.0072
Epoch 5/10
4347/4347 ━━━━━━━━━━━━━━━━━━━━ 195s 45ms/step - loss: 1.8455e-04 - mae: 0.0083 - val_loss: 1.5489e-04 - val_mae: 0.0072
Epoch 6/10
4347/4347 ━━━━━━━━━━━━━━━━━━━━ 148s 34ms/step - loss: 1.8342e-04 - mae: 0.0082 - val_loss: 1.5288e-04 - val_mae: 0.0072
Epoch 7/10
4347/4347 ━━━━━━━━━━━━━━━━━━━━ 147s 34ms/step - loss: 1.8248e-04 - mae: 0.0081 - val_loss: 1.6004e-04 - val_mae: 0.0077
Epoch 8/10
4347/4347 ━━━━━━━━━━━━━━━━━━━━ 148s 34ms/step - loss: 1.7800e-04 - mae: 0.00

In [162]:
def predict_stock(ticker, model, stock_embeddings, stock_tickers, sequence_length=30):
    # Fetch the recent 30 days of data for the ticker
    recent_data = yf.download(ticker, period="30d")
    recent_data = recent_data.dropna()
    
    # Scale the data
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(recent_data[['Open', 'High', 'Low', 'Close', 'Volume']])
    
    # Create the input sequence for prediction (last 30 days)
    input_sequence = scaled_data[-sequence_length:].reshape(1, sequence_length, 5)  # shape (1, 30, 5)
    
    # Ensure the stock embedding has the correct shape (1, embedding_dim)
    stock_index = stock_tickers.index(ticker)
    
    # Extract the embedding and check its size
    stock_embedding = stock_embeddings[stock_index]
    
    # Ensure that stock_embedding is a 1D array of size embedding_dim
    if stock_embedding.ndim == 1:
        stock_embedding = stock_embedding.reshape(1, -1)  # Reshape to (1, embedding_dim)
    
    # Print shapes for debugging
    print(f"input_sequence shape: {input_sequence.shape}")
    print(f"stock_embedding shape: {stock_embedding.shape}")
    
    # Predict the stock price (Close price)
    prediction = model.predict([input_sequence, stock_embedding])
    
    # De-normalize the prediction (map it back to the original scale)
    return scaler.inverse_transform([[0, 0, 0, prediction[0][0], 0]])[0][3]  # De-normalize the 'Close' price


In [163]:
stock_ticker_to_index = {ticker: idx for idx, ticker in enumerate(stock_tickers)}
stock_index = stock_ticker_to_index["COALINDIA.NS"]
stock_embedding = stock_embeddings[stock_index]


In [164]:
final = predict_stock("COALINDIA.NS", model, stock_embedding,final_ticker_list)
print(final)


[*********************100%%**********************]  1 of 1 completed


input_sequence shape: (1, 30, 5)
stock_embedding shape: ()


IndexError: tuple index out of range